In [1]:
import math
class DBN:
    def __init__(self, n_v, layers, k=1, mean_field=True):
        
        if n_v is None or layers is None: raise ValueError("Incorrect inputs for layer 0.")
        
        n_hs = [n_v]        
        n_layer = 0
        training_done = True
        rbms = []
        for (n_h, model) in layers:
            n_layer += 1
            if model is None:
                if n_h <= 0: raise ValueError("Incorrect inputs for layer %d" % (n_layer))
                else: n_hs.append(n_h)

                rbm = RBM(n_hs[n_layer-1], n_h, k=k)
                training_done = False

            else:
                assert training_done
                
                rbm = RBM.load_model(model)
                assert rbm.n_v == n_hs[n_layer-1]                
                n_hs.append(rbm.n_h)

            rbms.append(rbm)

        self.n_hs = n_hs
        self.n_layer = n_layer
        self.training_done = training_done
        self.rbms = rbms
        self.mean_field = mean_field
        return
    
    def forward(self, X):
        assert self.training_done
        
        Hp = X
        for i in range(self.n_layer):
            Hp, Hs = self.rbms[i].forward(Hp)
        
        return Hp, Hs

    def backward(self, H):
        assert self.training_done

        Vp = H
        for i in reversed(range(self.n_layer)):
            Vp, Vs = self.rbms[i].backward(Vp)
        
        return Vp, Vs

    def train_1layer_1batch(self, V, layer):
        Hp = V
        for i in range(layer):
            Hp, Hs = self.rbms[i].forward(Hp)
        
        if self.mean_field or layer==0: 
            self.rbms[layer].contrastive_divergence(Hp, self.learning)
        else:
            for j in range(5):
                Vs = np.random.binomial(1, Hp, size=Hp.shape)
                self.rbms[layer].contrastive_divergence(Vs, self.learning)
                
        return

    def train_1layer(self, X, layer, epochs, batch_size=10, learning=0.01, save_file=None):
        assert not self.rbms[layer].training_done
        
        self.learning = learning
        n_x = X.shape[0]
        n_batch = n_x//batch_size
        startrate = learning
        for e in range(epochs):
            learning = startrate * math.pow(0.1, e//50) 
            for i in range(n_batch):
                s = i*batch_size
                V = X[s:s+batch_size]
                self.train_1layer_1batch(V, layer)

        self.rbms[layer].save_model(save_file+"-"+str(layer+1)+".epochs"+str(epochs))
        self.rbms[layer].training_done = True
        return
    
    # this is internal API for more complex network to use
    def train_model(self, X, epochs=1, batch_size=10, learning=0.01, save_file=None):
        
        for layer in range(self.n_layer):
            if self.rbms[layer].training_done: continue
            self.train_1layer(X, layer, epochs, batch_size, learning, save_file)
        
        return

    # this is the API for app to use
    def train(self, X, epochs=1, batch_size=10, learning=0.01, save_file=None):
        if self.training_done: return
        
        save_file += ".dbn.layer" + str(self.n_layer)
        self.train_model(X, epochs, batch_size, learning, save_file)
        
        self.training_done = True
        return
    
    def reconstruct(self, X):
        h_layer = self.n_layer - 1
        Hp = X
        for i in range(h_layer):
            Hp, Hs = self.rbms[i].forward(Hp)
        
        Vp, Vs = self.rbms[h_layer].reconstruct(Hp)

        for i in reversed(range(h_layer)):
            Hp, Hs = self.rbms[i].backward(Hp)

        return Hp, Hs
    
    def show_features(self, shapes, title, count=-1):
        
        #for i in range(self.n_layer):
        #    rbm = self.rbms[i]
        #    rbm.show_features(shapes[i], "MNIST learned features in DBN layer %d" % (i+1), count)

        lower_rbm = None
        for i in range(self.n_layer):
            rbm = self.rbms[i]
            if i!=0:
                W = np.dot(rbm.W, lower_rbm.W)
                rbm = RBM(W.shape[1], W.shape[0], W=W)
               
            rbm.show_features(shapes[0], title +" learned features in layer %d" % (i+1), count)
            lower_rbm = rbm

        return lower_rbm

In [3]:
from sklearn.preprocessing import StandardScaler
import numpy as np
# Load your image data (for demonstration purposes, let's create a dummy 3D array)
# data = np.load('your_image_data.npy') # Your actual data loading code
data = np.random.rand(291, 275, 442)  # Dummy data with the same shape

class IR_DBN:
    
    def __init__(self, n_v, layers, data_path): 
        self.dbn = DBN(n_v, layers)
        
        data = np.load(f'data/data0.npy')
        # Reshape the data: (height * width, spectrum)
        nsamples, nx, ny = data.shape
        reshaped_data = data.reshape((nsamples*nx, ny))

        # Normalize the data
        scaler = StandardScaler()
        normalized_data = scaler.fit_transform(reshaped_data)
        self.train_input = normalized_data
        return

    def train(self, train_size=-1, n_epoch=1, batch_size=10, learning=0.01):
        X = []
        n_x = 0
        for x, y in self.train_input.read(train_size):
            X.append(x)
            n_x += 1

        X = np.array(X).reshape(n_x, -1) > 30
        X = X*1
        
        self.dbn.train(X, n_epoch, batch_size, learning, save_file="mnist")
        return
    
    def test_reconstruct(self, n):
        X=[]; i=2*n
        for x, y in self.test_input.read(n):
            x *= np.random.binomial(1, i/(n*2), size=(28,28))
            x = x * 2*n/i
            X.append(x)
            i -=1

        ncols = 10
        nrows = int(n/5)
        fig = plt.figure(figsize=(ncols, nrows), dpi=300)
        grid = Grid(fig, rect=111, nrows_ncols=(nrows,10))

        for i, ax in enumerate(grid):
            j = i//2
            if i%2 == 0:
                ax.imshow(X[j].reshape(28,28), cmap=mpl.cm.Greys)
                if i<ncols:
                    ax.set_title("Original")
            else:
                Vp, Vs = self.dbn.reconstruct(X[j].reshape(1, -1)/255)
                ax.imshow(Vp.reshape(28,28), cmap=mpl.cm.Greys)
                if i<ncols:
                    ax.set_title("Reconst.")

            ax.set_axis_off()

        fig.suptitle('Original and reconstructed digits side by side')
        fig.tight_layout()
        fig.subplots_adjust(top=0.80)    
        plt.show()        
        return

In [5]:
import sys

# Add the folder containing your module to the Python path
sys.path.append('DBN-master')

In [7]:
%run "/home/hakim/sciebo2/Uni/Semester 2/BioInfo/DBN-master/RBM.ipynb"

ModuleNotFoundError: No module named 'nbformat'